In [2]:
%load_ext lab_black

In [3]:
%cd /Users/anshilina/work/doc24

/Users/anshilina/work/doc24


In [3]:
import pandas as pd

# import os
import re
from tqdm import notebook

import requests
from bs4 import BeautifulSoup

In [4]:
PAGE_NUM = 0
PATH_TO_CITE = f"https://infopedia.su/12xba{PAGE_NUM}.html"
response = requests.get(f"{PATH_TO_CITE}")
soup = BeautifulSoup(response.content, "html.parser", from_encoding="utf-8")
# .select(".blink")

In [8]:
soup.find_all("p")

[<p><strong>КАТЕГОРИИ:</strong></p>,
 <p><strong>ТОП 10 на сайте</strong></p>,
 <p><strong>ЗНАЕТЕ ЛИ ВЫ?</strong></p>,
 <p> </p>,
 <p> </p>,
 <p> </p>,
 <p> </p>,
 <p><strong>II. ОСНОВНЫЕ ЖАЛОБЫ БОЛЬНОГО И ИХ ДЕТАЛИЗАЦИЯ</strong></p>,
 <p>(жалобы больного при поступлении)</p>,
 <p>Необходимо подробно изложить все жалобы больного, кото­рые касаются заболевания, по поводу которого больной обратился к врачу.</p>,
 <p>Для выяснения жалоб больному задают вопрос: что Вас бес­покоит?, на что Вы жалуетесь? И дают больному возможность свободно рассказать о своих ощущениях, переживаниях. В про­цессе ответа необходимо тактично корректировать беседу наводя­щими вопросами, уточнениями, одновременно делая заметки в виде черновых записей. Если больной очень разговорчив и излагает, не имеющие непосредственного отношения к жалобам, то не­обходимо путем деликатной осторожно коррекции придать бесе­де целенаправленную форму. Если больной излагает жалобы не­достаточно полно, то нужно выяснить более настойч

_____

In [462]:
PATH_TO_CITE = f"https://www.vidal.ru/drugs/products/p/rus-a"
response = requests.get(f"{PATH_TO_CITE}")
soup = BeautifulSoup(response.content, "html.parser")

paths_to_cite = []
for i in range(2):
    num_pages = len(soup.find_all("div", class_="letters-russian")[i].find_all("a"))
    for ind in notebook.tqdm(range(num_pages)):
        page_link = soup.find_all("div", class_="letters-russian")[i].find_all("a")[
            ind
        ]["href"]
        paths_to_cite.append("https://www.vidal.ru" + page_link)

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
PATH_TO_CITE = f"https://www.vidal.ru/drugs/products/p/rus-a"
response = requests.get(f"{PATH_TO_CITE}")
soup = BeautifulSoup(response.content, "html.parser")

In [512]:
def get_med_links(path_to_cite):
    response = requests.get(f"{path_to_cite}")
    soup = BeautifulSoup(response.content, "html.parser")

    links = []
    num_products = len(soup.find_all("td", class_="products-table-name"))
    for ind in range(num_products):
        links.append(
            soup.find_all("td", class_="products-table-name")[ind].find_all(
                "a", class_="no-underline"
            )[0]["href"]
        )

    page_num = 2
    while running:
        PATH_TO_CITE = f"{path_to_cite}?p={page_num}"
        response = requests.get(f"{PATH_TO_CITE}")
        soup = BeautifulSoup(response.content, "html.parser")
        page_num += 1

        num_products = len(soup.find_all("td", class_="products-table-name"))
        if num_products == 0:
            break
        for ind in range(num_products):
            link = soup.find_all("td", class_="products-table-name")[ind].find_all(
                "a", class_="no-underline"
            )[0]["href"]
            links.append(link)
    return links

In [518]:
df = []
for path in notebook.tqdm(paths_to_cite[:-5]):
    letter = re.findall(
        r"https://www.vidal.ru/drugs/products/p/[rus-]*(\w+|\d)", path,
    )[0]
    print(f'Лекарства на букву "{letter}"')
    df += get_med_links(path)

  0%|          | 0/28 [00:00<?, ?it/s]

Лекарства на букву "a"
Лекарства на букву "b"
Лекарства на букву "v"
Лекарства на букву "g"
Лекарства на букву "d"
Лекарства на букву "e"
Лекарства на букву "zh"
Лекарства на букву "z"
Лекарства на букву "i"
Лекарства на букву "j"
Лекарства на букву "k"
Лекарства на букву "l"
Лекарства на букву "m"
Лекарства на букву "n"
Лекарства на букву "o"
Лекарства на букву "p"
Лекарства на букву "r"
Лекарства на букву "s"
Лекарства на букву "t"
Лекарства на букву "u"
Лекарства на букву "f"
Лекарства на букву "h"
Лекарства на букву "ts"
Лекарства на букву "ch"
Лекарства на букву "h"
Лекарства на букву "eh"
Лекарства на букву "yu"
Лекарства на букву "ya"


In [534]:
# with open("links.txt", "w") as file:
#     for row in df:
#         s = "".join(map(str, row))
#         file.write(s + "\n")

In [4]:
my_file = open("links.txt", "r")

# reading the file
data = my_file.readlines()
data = [i.strip() for i in data]

In [5]:
len(data)

21947

In [8]:
results = []

for cite in notebook.tqdm(data[:10]):
    PATH_TO_CITE = f"https://www.vidal.ru{cite}"
    response = requests.get(PATH_TO_CITE)
    soup = BeautifulSoup(response.content, "html.parser")
    try:
        if "Показания" in (
            soup.find_all("div", class_="block")[9]
            .find_all("h2", class_="block-head")[0]
            .text.replace("\n", "")
            .replace("   ", "")
            .replace("  ", " ")
            .strip()
        ):
            name = (
                soup.find_all("div", class_="block")[9]
                .find_all("h2", class_="block-head")[0]
                .text.replace("\n", "")
                .replace("   ", "")
                .replace("  ", " ")
                .strip()
            )
            description = (
                soup.find_all("div", class_="block")[9]
                .find_all("div", class_="block-content")[0]
                .text.strip()
            )
        elif "Показания" in (
            soup.find_all("div", class_="block")[8]
            .find_all("h2", class_="block-head")[0]
            .text.replace("\n", "")
            .replace("   ", "")
            .replace("  ", " ")
            .strip()
        ):
            name = (
                soup.find_all("div", class_="block")[8]
                .find_all("h2", class_="block-head")[0]
                .text.replace("\n", "")
                .replace("   ", "")
                .replace("  ", " ")
                .strip()
            )
            description = (
                soup.find_all("div", class_="block")[8]
                .find_all("div", class_="block-content")[0]
                .text.strip()
            )

        else:
            name = (
                soup.find_all("div", class_="block")[7]
                .find_all("h2", class_="block-head")[0]
                .text.replace("\n", "")
                .replace("   ", "")
                .replace("  ", " ")
                .strip()
            )
            description = (
                soup.find_all("div", class_="block")[7]
                .find_all("div", class_="block-content")[0]
                .text.strip()
            )

    except:
        name = None

        description = None

    results.append((name, description, PATH_TO_CITE))

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
data = pd.DataFrame(results, columns=["название", "показание", "ссылка на источник"])
data

,название,показание,ссылка на источник
0,Показания активных веществ препарата аАКДС-Геп...,"Вакцина для профилактики дифтерии, столбняка, ...",https://www.vidal.ru/drugs/acdt-hep-b-hib-aapd...
1,Показания препарата Абаджио®,лечение взрослых пациентов с рецидивирующе-рем...,https://www.vidal.ru/drugs/aubagio__42816
2,Показания активных веществ препарата Абакавир,Лечение ВИЧ-инфекции (в составе комбинированно...,https://www.vidal.ru/drugs/abacavir-17
3,Показания активных веществ препарата Абакавир,Лечение ВИЧ-инфекции (в составе комбинированно...,https://www.vidal.ru/drugs/abacavir-4
4,Показания активных веществ препарата Абакавир,Лечение ВИЧ-инфекции (в составе комбинированно...,https://www.vidal.ru/drugs/abacavir-10
...,...,...,...
12086,Показания активных веществ препарата Нафтизин,"Для интраназального применения: острый ринит, ...",https://www.vidal.ru/drugs/naphthyzin__9907
12087,Показания активных веществ препарата Нафтизин,"Для интраназального применения: острый ринит, ...",https://www.vidal.ru/drugs/naphthizin
12088,Показания активных веществ препарата Нафтизин,"Для интраназального применения: острый ринит, ...",https://www.vidal.ru/drugs/naphthyzin-11
12089,None,None,https://www.vidal.ru/drugs/naphthyzin-4


_____

In [10]:
data = pd.read_csv("data_1.csv")

In [11]:
data[["название", "показание"]].duplicated().sum()

5726

In [22]:
data[["название", "показание"]].drop_duplicates()

,название,показание
0,Показания активных веществ препарата аАКДС-Геп...,"Вакцина для профилактики дифтерии, столбняка, ..."
1,Показания препарата Абаджио®,лечение взрослых пациентов с рецидивирующе-рем...
2,Показания активных веществ препарата Абакавир,Лечение ВИЧ-инфекции (в составе комбинированно...
17,Показания активных веществ препарата Абакавир ...,Лечение ВИЧ-инфекции (в составе комбинированно...
18,Показания активных веществ препарата Абакавир-АВС,Лечение ВИЧ-инфекции (в составе комбинированно...
...,...,...
12077,Показания препарата Натулан,болезнь Ходжкина (лимфогранулематоз);неходжкин...
12078,Режим дозирования,Дозировка зависит от стадии заболевания и не з...
12079,Показания активных веществ препарата Натуролакс,Острый и хронический запор; анальные трещины; ...
12080,Показания препарата Нафтадерм®,псориаз;экзема;нейродермит;себорея;розовый лиш...
